Observação: Apesar das várias tentativas, não conseguimos executar este notebook por mais de três épocas devido à falta de recurso computacional. No entanto, para apenas três épocas, a metodologia apresentou resultados interessantes.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Reshape, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications.resnet50 import preprocess_input
import cv2
import random
import pickle
from sklearn.metrics import confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Caminhos das pastas contendo as imagens de treinamento e validação
train_data_dir = "/content/drive/Shareddrives/IA901/dados melanoma/data/interim4/interim4/treino"
validation_data_dir = "/content/drive/Shareddrives/IA901/dados melanoma/data/data/interim/validacao"

batch_size = 4

def preprocess_image(image):
    standardized_image = preprocess_input(image)
    return standardized_image

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    #width_shift_range=0.3,
    height_shift_range=0.1,
    #shear_range=0.1,
    #zoom_range=0.1,
    #brightness_range=[0.7, 1.3],
    fill_mode='nearest',
    preprocessing_function=preprocess_image,
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, train_generator.num_classes), dtype=tf.float32)
    )
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    #width_shift_range=0.3,
    height_shift_range=0.1,
    #shear_range=0.1,
    #zoom_range=0.1,
    #brightness_range=[0.7, 1.3],
    fill_mode='nearest',
    preprocessing_function=preprocess_image

)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, validation_generator.num_classes), dtype=tf.float32)
    )
)

train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 48456 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Found 8608 images belonging to 2 classes.


In [7]:
# Define a arquitetura do modelo
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    weights='imagenet',
    include_top=False
)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(2, activation='sigmoid')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Congela todas as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False
tf.config.run_functions_eagerly(True)  # Otherwise error

# Callbacks
cb_early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=300)

# Define o caminho para salvar os checkpoints
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/models/checkpoints/mobilenet_oversampling.{epoch:02d}.h5"

# Callback para salvar os checkpoints
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

callbacks_list = [checkpoint_callback, cb_early_stopper]

# Define o otimizador como AdamW com os pesos de classe
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[tf.keras.metrics.Recall(), 'AUC'])

# Verifica se há um checkpoint existente para carregar
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_path)
if latest_checkpoint is not None:
    print("Carregando checkpoint existente:", latest_checkpoint)
    model.load_weights(latest_checkpoint)

In [ ]:
history = model.fit(
    train_dataset,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=5,
    validation_data=validation_dataset,
    callbacks=callbacks_list,
    validation_steps=validation_generator.n // batch_size
)

Epoch 1/5
  719/12114 [>.............................] - ETA: 8:11:15 - loss: 0.5422 - recall_1: 0.7138 - auc: 0.7957